# Simulate BGs
* description: simulate bgs given common user input
* version: 0.0.10
* created: 2018-12-15
* author: Ed Nykaza 
* dependencies:
    * requires conda environment (see readme for instructions)
    * OR you can run directly from binder
* license: BSD-2-Clause

## General form of simulation equation
Delta BG is the change in blood glucose (BG) with the units: $\frac{(mg/dL)}{5min}$

$\Delta BG[t] = EGP[t]+ \Delta BG_{Carb}[t]+\Delta BG_{Insulin}$

where EGP is the endogenous glucose production, which can be thought of as a surrogate for the basal rate as the purpose of the basal rate is to counter the EGP. Also, given basal rates are givin U/hr, we must divide by 12 to get basalRate in proper units.

### The equation can be expanded to include common insulin pump therapy parameters:

$\Delta BG[t] = ISF \bigg(\frac{BR[t]}{12} + \frac{ACV[t]}{CIR[t]} - AIV[t] \bigg)$

where:

* $ACV[t] = \sum CA[t]*CAC[t]$ <br />
* $AIV[t] = \sum IA[t]*IAC[t]$ <br />
* $ISF$ is the insulin sensitivity factor (mg/dL) / U <br />
* $BR$ is the basal rate U / hr <br />
* $CIR$ is the carb-to-insulin ratio g / U <br />
* $ACV$ is the active carb velocity g / 5min <br />
* $CA$ is each carb amount g  <br />
* $CAC$ is each carb activity curve  1 / 5min  <br />
* $AIV$ is the active insulin velocity U / 5 min <br />
* $IA$ is each insulin amount g  <br />
* $IAC$ is each insulin activity curve  1 / 5min  <br />

## Instructions

Run the three cells below to interact with the simulation below. Recall that you press  `shift-return` to execute each cell, or you can press the `Run` button (above)


In [16]:
# %% 1. LOAD REQUIRED LIBRARIES
%matplotlib inline
import pdb
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import plotly.plotly as py
import plotly.figure_factory as ff
from ipywidgets import interact, interactive, fixed, interact_manual, link, Layout, Button, Box
from IPython.display import clear_output
import ipywidgets as widgets

# version number should match the version in the header
majorVersion = 0
minorVersion = 0
microVersion = 10
versionNumber = "v-%s%s%s"%(majorVersion, minorVersion, microVersion)

# Save the output figure (True/False)
saveFigure = False


In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [39]:
# currentTime (can take string or date time format)
currentTime = "12:00 PM"  # dt.datetime.now()

# starting BG Level (valid range is 40 to 400 mg/dL)
startingBGLevel = 180

# Insulin Model (4 options)
insulinModel = "adult"  # "adult", "child", "fiasp", or "walsh"


# %% GLOBAL FIGURE PROPERTIES
bgRange = [35, 405]
figureSizeInches = (15, 7*2)
coord_color = "#c0c0c0"
xLabel = "Time Relative to Now (Minutes)"
yLabel = "Glucose (mg/dL)"
labelFontSize = 15
tickLabelFontSize = 12
legendFontSize = 10

# Save the output figure (True/False)
saveFigure = False

if saveFigure:
    figureName = currentTime.strftime("%Y-%m-%d-%H-%M") + "-simulate-" + versionNumber
    outputPath = os.path.join(".", "figures")
    # create output folder if it doesn't exist
    if not os.path.isdir(outputPath):
        os.makedirs(outputPath)

# if the Roboto font exists in specified folder, else use the system default font 
figureFontNamePath = os.path.join(".", "Roboto-Bold.ttf")
if os.path.exists(figureFontNamePath):
    figureFont = fm.FontProperties(fname=figureFontNamePath)
else:
    figureFont = fm.FontProperties()

# font size
font = {'weight': 'bold',
        'size': labelFontSize}
plt.rc('font', **font)
plt.rcParams["axes.labelweight"] = "bold"

## %% MAKE INPUT INTERFACE
insulinModel = "adult"
if insulinModel == "adult":
    isfMin, isfMax, isfStep, isfStart = 10, 100, 1, 50

elif insulinModel == "child":
    isfMin, isfMax, isfStep, isfStart = 40, 400, 1, 50

else:  # this is the full range
    isfMin, isfMax, isfStep, isfStart = 10, 400, 1, 50

# Labels
layout=Layout(width='97%')
dataTypeWid=widgets.Dropdown(
    options=["Insulin Sensitivity Factor", "Carb-to-Insulin Ratio",
             "Scheduled Basal Rate", "Temp Basal Events", "Bolus Events"],
    value="Insulin Sensitivity Factor",
    description="",
    disabled=False,
    layout=layout
)

typeLabel = widgets.Label("Select a Type", layout=layout)
typeBox = widgets.VBox([typeLabel, dataTypeWid])

hourListFull = ["12:00 AM", "12:30 AM", "1:00 AM", "1:30 AM",
             "2:00 AM", "2:30 AM", "3:00 AM", "3:30 AM",
             "4:00 AM", "4:30 AM", "5:00 AM", "5:30 AM",
             "6:00 AM", "6:30 AM", "7:00 AM", "7:30 AM",
             "8:00 AM", "8:30 AM", "9:00 AM", "9:30 AM",
             "10:00 AM", "10:30 AM", "11:00 AM", "11:30 AM",
             "12:00 PM", "12:30 PM", "1:00 PM", "1:30 PM",
             "2:00 PM", "2:30 PM", "3:00 PM", "3:30 PM",
             "4:00 PM", "4:30 PM", "5:00 PM", "5:30 PM",
             "6:00 PM", "6:30 PM", "7:00 PM", "7:30 PM",
             "8:00 PM", "8:30 PM", "9:00 PM", "9:30 PM",
             "10:00 PM", "10:30 PM", "11:00 PM", "11:30 PM"
            ]

hourList = ["12:00 AM", "1:00 AM", 
             "2:00 AM", "3:00 AM", 
             "4:00 AM", "5:00 AM", 
             "6:00 AM", "7:00 AM", 
             "8:00 AM", "9:00 AM", 
             "10:00 AM", "11:00 AM", 
             "12:00 PM", "1:00 PM", 
             "2:00 PM", "3:00 PM",
             "4:00 PM", "5:00 PM",
             "6:00 PM", "7:00 PM",
             "8:00 PM", "9:00 PM", 
             "10:00 PM", "11:00 PM"
            ]

hourListLabels = [
    "12AM", "1AM", 
    "2AM", "3AM", 
    "4AM", "5AM", 
    "6AM", "7AM", 
    "8AM", "9AM", 
    "10AM", "11AM", 
    "12PM", "1PM", 
    "2PM", "3PM",
    "4PM", "5PM",
    "6PM", "7PM",
    "8PM", "9PM", 
    "10PM", "11PM", "12AM"
]

isfLabel = r'Insulin Sensitivity Factor $\frac{mg/dL}{U}$'
cirLabel = r'Carb-to-Insulin Ratio $\frac{g}{U}$'
sbrLabel = r'Scheduled Basal Rate $\frac{U}{hr}$'
abrLabel = r'Temporary Basal Rates $\frac{U}{hr}$'
carbLabel = r'Carbs Consumed ${g}$'
insulinLabel = r'Insulin Given ${U}$'


timeWid1=widgets.Dropdown(
    options=hourListFull,
    value="12:00 AM",
    description="",
    disabled=False,
    layout=layout
)

timeWid2=widgets.Dropdown(
    options=hourListFull,
    value="12:00 AM",
    description="",
    disabled=False,
    layout=Layout(width='97%', visibility="hidden")
)

timeLabel = widgets.Label("Select a Time")
timeBox = widgets.VBox([timeLabel, timeWid1, timeWid2])

valueWid1=widgets.FloatSlider(
    value=isfStart,
    min=isfMin,
    max=isfMax,
    step=isfStep,
    description="",
    disabled=False,
    readout_format='.0f',
    continuous_update=False)

valueWid2=widgets.FloatSlider(
    value=1,
    min=0.025,
    max=10,
    step=0.025,
    description="",
    disabled=False,
    readout_format='.3f',
    continuous_update=False,
    layout=Layout(visibility="hidden"))

valueWid1Label = widgets.Label(isfLabel)
valueWid1Box = widgets.VBox([valueWid1Label, valueWid1, valueWid2])

durationWid1=widgets.Dropdown(
    options=np.arange(0, 24*60, 5),
    value=0,
    description="",
    disabled=False,
    layout=layout
)

durationWid2=widgets.Dropdown(
    options=np.arange(0, 24*60, 5),
    value=0,
    description="",
    disabled=False,
    layout=Layout(visibility="hidden", width='97%')
)

durationWid1Label = widgets.Label("Duration (min)")
durationWid1Box = widgets.VBox([durationWid1Label, durationWid1, durationWid2],
                              layout=Layout(visibility="hidden"))

addResetButton=widgets.ToggleButtons(
    options=["Add", "Reset"],
    value="Add",
    description='',
    disabled=False,
    button_style="",
    layout=Layout(width='20%'),
    continuous_update=False
)

addResetLabel = widgets.Label("Add or Reset")
addResetBox = widgets.VBox([addResetLabel, addResetButton])

inputBox = widgets.HBox([typeBox, timeBox, valueWid1Box, durationWid1Box, addResetBox],
             layout=Layout(
                 display='flex',
                 flex_flow='row',
                 justify_content="space-around",
                 #align_items='center',
                 #align_content="space-around",
                 width='100%'
             )
)

def log_10_product(x, pos):
    if x < 0.025:
        return '0'
    if x < .1:
        return '%.3f' % (x)    
    if x < 1:
        return '%.2f' % (x)
    if x < 10:
        return '%.1f' % (x)
    if x >= 10:
        return '%1i' % (x)


def common_figure_elements(ax, xLabel, xLabelFontSize, tickLabelFontSize, coord_color):
    # x-axis items
    ax.set_xlabel(xLabel, fontsize=xLabelFontSize, color=coord_color)

    # define the spines and grid
    ax.spines['bottom'].set_color(coord_color)
    ax.spines['top'].set_color(coord_color)
    ax.spines['left'].set_color(coord_color)
    ax.spines['right'].set_color(coord_color)
    ax.spines['bottom'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.grid(ls='-', color=coord_color)

    # set size of ticklabels
    ax.tick_params(axis='both', labelsize=tickLabelFontSize, colors=coord_color)

    return ax


# %% first create a continuguous time series
startTime = pd.to_datetime("12:00 AM")
endTime = pd.to_datetime("11:55 PM")
rng = pd.date_range(startTime, endTime, freq=("5min"))
df = pd.DataFrame(rng, columns=["dateTime"])
df["minutesRelativeToNow"] = np.arange(-12*60, 12*60, 5)

# Insulin Sensitivity Factor (mg/dL)/U)
isfDF = pd.DataFrame(np.array([["12:00 AM", 40]]), columns=["isftime", "isf"])
isfDF["dateTime"] = pd.to_datetime(isfDF["isftime"])
isfDF["isf"] = isfDF["isf"].astype(float)
df = pd.merge(df, isfDF, how="left", on="dateTime")
df["isf"] = df["isf"].astype(float)

# Carb-to-Insulin Ratio (g/U)
cirDF = pd.DataFrame(np.array([["12:00 AM", 10]]), columns=["cirtime", "cir"])
cirDF["dateTime"] = pd.to_datetime(cirDF["cirtime"])
cirDF["cir"] = cirDF["cir"].astype(float)
df = pd.merge(df, cirDF, how="left", on="dateTime")
df["cir"] = df["cir"].astype(float)

# Scheduled Basal Rates (time, basalRate [U/hr])
sbrDF = pd.DataFrame(np.array([["12:00 AM", 1]]), columns=["sbrtime", "sbr"])
sbrDF["dateTime"] = pd.to_datetime(sbrDF["sbrtime"])
sbrDF["sbr"] = sbrDF["sbr"].astype(float)
df = pd.merge(df, sbrDF, how="left", on="dateTime")
df["sbr"] = df["sbr"].astype(float)
df.fillna(method='ffill', inplace=True) 

# Temp Basal Events (including suspends) (time, basalRate, duration [U/hr])
abrDF = pd.DataFrame(np.array([["12:00 AM", 1, 0]]), columns=["abrtime", "abr", "abrDuration"])
abrDF["dateTime"] = pd.to_datetime(abrDF["abrtime"])
abrDF["abr"] = abrDF["abr"].astype(float)
abrDF["abrDuration"] = abrDF["abrDuration"].astype(float)
df = pd.merge(df, abrDF, how="left", on="dateTime")
df["abr"] = df["abr"].astype(float)
df["abrDuration"] = df["abrDuration"].astype(float)
#df.fillna(method='ffill', inplace=True) 

# Carb Events (carbtime, carb, carbModel, insulintime, insulinAmount)
carbDF = pd.DataFrame(np.array([["12:00 AM", 0, "lowGI"]]),
                      columns=["carbtime", "carb", "carbModel"])

# make sure the values are floats and not integers
carbDF["dateTime"] = pd.to_datetime(carbDF["carbtime"])
carbDF["carb"] = carbDF["carb"].astype(float)
df = pd.merge(df, carbDF, how="left", on="dateTime")
df["carb"] = df["carb"].astype(float)

# Insulin (Bolus) Events (insulintime, insulin, insulintime, insulinAmount)
insulinDF = pd.DataFrame(np.array([["12:00 AM", 0, "adult"]]),
                          columns=["insulintime", "insulin", "insulinModel"])

# make sure the values are floats and not integers
insulinDF["dateTime"] = pd.to_datetime(insulinDF["insulintime"])
insulinDF["insulin"] = insulinDF["insulin"].astype(float)
df = pd.merge(df, insulinDF, how="left", on="dateTime")
df["insulin"] = df["insulin"].astype(float)

previousType = dataTypeWid.options[0]

# %% FUNCTIONS
def create_deltaBG_functions(deltaBgEquation):

    def get_deltaBG_function(ISF=0, BR=0, TB=0,
                             ACV=0, CIR=1, ABOLV=0,
                             AIV=0, deltaBgEquation=deltaBgEquation):

        return eval(deltaBgEquation)

    return get_deltaBG_function


def set_values(vDF, vAbr, vLabel, vMin, vMax, vStep, vFormat):
    valueWid1Label.value = vLabel
    if valueWid1.max < vMin:
        valueWid1.max = vMax
        valueWid1.min = vMin
    else:
        valueWid1.min = vMin
        valueWid1.max = vMax
    valueWid1.step = vStep
    valueWid1.readout_format = vFormat
    timeWid1.value = vDF[vAbr + "time"].max()
    valueWid1.value = vDF.loc[vDF[vAbr + "time"] == vDF[vAbr + "time"].max(), vAbr]
    durationWid1Box.layout.visibility="hidden"
    durationWid2.layout.visibility="hidden"
    timeWid2.layout.visibility="hidden"
    valueWid2.layout.visibility="hidden"
    valueWid1.description=""
    valueWid2.description=""
    timeWid1.disabled = False
    valueWid1.disabled = False
    timeWid2.disabled = False
    valueWid1.disabled = False

    return


def set_values2(vDF, vAbr, vMin, vMax, vStep, vFormat):
    if valueWid2.max < vMin:
        valueWid2.max = vMax
        valueWid2.min = vMin
    else:
        valueWid2.min = vMin
        valueWid2.max = vMax
    valueWid2.step = vStep
    valueWid2.readout_format = vFormat
    timeWid2.value = vDF[vAbr + "time"].max()
    valueWid2.value = vDF.loc[vDF[vAbr + "time"] == vDF[vAbr + "time"].max(), vAbr]
    return


def add_values(mDF, vDF, vAbr):
    newDataDF = pd.DataFrame(np.array([[timeWid1.value, valueWid1.value]]), columns=[vAbr + "time", vAbr])
    newDataDF[vAbr] = newDataDF[vAbr].astype(float)
    if ((len(vDF) == 1) & (vDF.loc[0, vAbr] == 0)):
        vDF = newDataDF
        vDF[vAbr] = vDF[vAbr].astype(float)
    else:
        vDF = pd.concat([vDF, newDataDF], ignore_index=True, sort=True)
        vDF.drop_duplicates(subset=vAbr + "time", keep='last', inplace=True)
        vDF.reset_index(drop=True, inplace=True)

    vDF["dateTime"] = pd.to_datetime(vDF[vAbr + "time"])
    mDF.drop(columns=[vAbr, vAbr + "time"], inplace=True)
    mDF = pd.merge(mDF, vDF, how="left", on="dateTime")
    mDF.drop_duplicates(subset="dateTime", keep='last', inplace=True)
    mDF[vAbr] = mDF[vAbr].astype(float)
    mDF[vAbr].fillna(method='ffill', inplace=True)

    return mDF, vDF


def add_carbValues(mDF, vDF, vAbr):
    newDataDF = pd.DataFrame(np.array([[timeWid2.value, valueWid2.value, durationWid2.value]]),
                             columns=[vAbr + "time", vAbr, "carbModel"])
    newDataDF[vAbr] = newDataDF[vAbr].astype(float)
    if ((len(vDF) == 1) & (vDF.loc[0, vAbr] == 0)):
        vDF = newDataDF
        vDF[vAbr] = vDF[vAbr].astype(float)
    else:
        vDF = pd.concat([vDF, newDataDF], ignore_index=True, sort=True)
        vDF.drop_duplicates(subset=vAbr + "time", keep='last', inplace=True)
        vDF.reset_index(drop=True, inplace=True)

    vDF["dateTime"] = pd.to_datetime(vDF[vAbr + "time"])
    mDF.drop(columns=[vAbr, vAbr + "time", vAbr + "Model"], inplace=True)
    mDF = pd.merge(mDF, vDF, how="left", on="dateTime")
    mDF.drop_duplicates(subset="dateTime", keep='last', inplace=True)
    mDF[vAbr] = mDF[vAbr].astype(float)
    return mDF, vDF


def add_insulinValues(mDF, vDF, vAbr):
    newDataDF = pd.DataFrame(np.array([[timeWid1.value, valueWid1.value, insulinModel]]),
                             columns=[vAbr + "time", vAbr, "insulinModel"])
    newDataDF[vAbr] = newDataDF[vAbr].astype(float)
    if ((len(vDF) == 1) & (vDF.loc[0, vAbr] == 0)):
        vDF = newDataDF
        vDF[vAbr] = vDF[vAbr].astype(float)
    else:
        vDF = pd.concat([vDF, newDataDF], ignore_index=True, sort=True)
        vDF.drop_duplicates(subset=vAbr + "time", keep='last', inplace=True)
        vDF.reset_index(drop=True, inplace=True)

    vDF["dateTime"] = pd.to_datetime(vDF[vAbr + "time"])
    mDF.drop(columns=[vAbr, vAbr + "time", vAbr + "Model"], inplace=True)
    mDF = pd.merge(mDF, vDF, how="left", on="dateTime")
    mDF.drop_duplicates(subset="dateTime", keep='last', inplace=True)
    mDF[vAbr] = mDF[vAbr].astype(float)
    return mDF, vDF


def add_valuesAndDuration(mDF, vDF, vAbr):
    newDataDF = pd.DataFrame(np.array([[timeWid1.value, valueWid1.value, durationWid1.value]]),
                             columns=[vAbr + "time", vAbr, vAbr + "Duration"])
    newDataDF[vAbr] = newDataDF[vAbr].astype(float)
    newDataDF[vAbr + "Duration"] = newDataDF[vAbr + "Duration"].astype(float)
    if ((len(vDF) == 1) & (vDF.loc[0, vAbr] == 0)):
        vDF = newDataDF
        vDF[vAbr] = vDF[vAbr].astype(float)
        vDF[vAbr + "Duration"] = vDF[vAbr + "Duration"].astype(float)
    else:
        vDF = pd.concat([vDF, newDataDF], ignore_index=True, sort=True)
        vDF.drop_duplicates(subset=vAbr + "time", keep='last', inplace=True)
        vDF.reset_index(drop=True, inplace=True)

    vDF["dateTime"] = pd.to_datetime(vDF[vAbr + "time"])
    
    abr = pd.DataFrame()
    for i in range(len(vDF)):
        startTime = vDF.loc[i, "dateTime"]
        endTime = startTime + pd.Timedelta(np.int(vDF.loc[i, "abrDuration"]), unit="m")
        rng = pd.date_range(startTime, endTime, freq="5min")
        tempDf = pd.DataFrame(rng[:-1], columns=["dateTime"])
        tempDf["abr"] = vDF.loc[i, "abr"]
        tempDf["abrtime"] = vDF.loc[i, "abrtime"]
        tempDf["abrDuration"] = vDF.loc[i, "abrDuration"]
        abr = pd.concat([abr, tempDf], ignore_index=True)
    
    mDF.drop(columns=[vAbr, vAbr + "time", vAbr + "Duration"], inplace=True)
    mDF = pd.merge(mDF, abr, how="left", on="dateTime")
    mDF.drop_duplicates(subset="dateTime", keep='last', inplace=True)
    mDF[vAbr] = mDF[vAbr].astype(float)
    mDF[vAbr + "Duration"] = mDF[vAbr + "Duration"].astype(float)
    return mDF, vDF    


def reset_values(vValue=np.nan, vValue2=np.nan):
    if pd.notnull(vValue):
        valueWid1.value = vValue
        timeWid1.value = "12:00 AM"
    
    if pd.notnull(vValue2):
        valueWid2.value = vValue2
        timeWid2.value = "12:00 AM"
    addResetButton.value = "Add"
    return 

def f(a, b, c, d, g, h, j, k):
    global df, isfDF, cirDF, sbrDF, abrDF, carbDF, insulinDF
    global insulinModel, previousType, saveFigure, figureSizeInches

    # set the min, max, step, and starting values depending on the insulin model
    if insulinModel == "adult":
        isfMin, isfMax, isfStep, isfStart = 10, 100, 1, 50
        cirMin, cirMax, cirStep, cirStart = 4, 24, 1, 10
        sbrMin, sbrMax, sbrStep, sbrStart = 0.1, 3, 0.025, 1
        abrMin, abrMax, abrStep, abrStart = 0, 12, 0.025, 1
     
    elif insulinModel == "child":
        isfMin, isfMax, isfStep, isfStart = 40, 400, 1, 50
        cirMin, cirMax, cirStep, cirStart = 5, 40, 1, 10
        sbrMin, sbrMax, sbrStep, sbrStart = 0.025, 2, 0.025, 0.5  
        abrMin, abrMax, abrStep, abrStart = 0, 8, 0.025, 1
        
    else:  # this is the full range
        isfMin, isfMax, isfStep, isfStart = 10, 400, 1, 50
        cirMin, cirMax, cirStep, cirStart = 4, 40, 1, 10
        sbrMin, sbrMax, sbrStep, sbrStart = 0.025, 6, 0.025, 1
        abrMin, abrMax, abrStep, abrStart = 0, 12, 0.025, 1

    carbMin, carbMax, carbStep, carbStart = 0, 100, 1, 0
    insulinMin, insulinMax, insulinStep, insulinStart = 0, 10, 0.025, 0

    # set up the figure and subplots
    fig = plt.figure(figsize=figureSizeInches)
    widths = [1, 1, 1, 1]
    heights = [4, 3, 1, 2]
    spec = fig.add_gridspec(ncols=4, nrows=4, width_ratios=widths, height_ratios=heights)
    
    ax0 = fig.add_subplot(spec[0, :])
    ax0.set_xlim([-735, 735])
    ax = fig.add_subplot(spec[1, :], sharex=ax0)
    ax5 = fig.add_subplot(spec[2, :], sharex=ax0)
    ax7 = fig.add_subplot(spec[3, :], sharex=ax0)
    
    ax0 = common_figure_elements(ax0, "", labelFontSize, tickLabelFontSize, coord_color)
    ax0.set_xticks(np.arange(-720, 720 + 60, 60))
    ax0.set_ylim([0.01, 500])
    ax0.set_yscale('log')
    formatter = ticker.FuncFormatter(log_10_product)
    ax0.yaxis.set_major_formatter(formatter)
    ax0.set_yticks([0.015, 0.025, 0.05, 0.1, 0.25, 0.5, 1, 2.5, 5, 10, 25, 50, 100, 250, 500])
    ax0.tick_params(axis='y', which='minor', left=False)
    ax0.set_title("Settings & Events", color=coord_color)
    ax0.set_xticklabels(hourListLabels, visible=False)
    
    ax = common_figure_elements(ax, "", labelFontSize, tickLabelFontSize, coord_color)
    ax.set_title("Simulated Blood Glucose (BG)", color=coord_color)
    ax.set_xticklabels(hourListLabels, visible=False)
    ax.set_ylim(bgRange)
    ax.set_yticks(np.arange(40, 440, step=40))

    ax5 = common_figure_elements(ax5, "", labelFontSize, tickLabelFontSize, coord_color)
    ax5.set_title("Effective Basal Rate", color=coord_color)
    ax5.set_xticklabels(hourListLabels, visible=False)
    
    ax7 = common_figure_elements(ax7, "", labelFontSize, tickLabelFontSize, coord_color)
    ax7.set_title("Change in Blood Glucose (Delta BG)", color=coord_color)
    ax7.set_xticklabels(hourListLabels, visible=True, rotation=45)
    
    # logic for each data type
    if "Insulin Sensitivity Factor" in a:
        if previousType != a:
            set_values(isfDF,  "isf", isfLabel, isfMin, isfMax, isfStep, '.0f')

        if "Add" in d:
            df, isfDF = add_values(df, isfDF, "isf")
            
        if "Reset" in d:
            isfDF = pd.DataFrame(np.array([["12:00 AM", 0]]), columns=["isftime", "isf"])
            isfDF["isf"] = isfDF["isf"].astype(float)
            reset_values(isfStart)            
    
    if "Carb-to-Insulin Ratio" in a:
        if previousType != a:
            set_values(cirDF,  "cir", cirLabel,
                       cirMin, cirMax, cirStep, '.0f')
        if "Add" in d:
            df, cirDF = add_values(df, cirDF, "cir")
            
        if "Reset" in d:
            cirDF = pd.DataFrame(np.array([["12:00 AM", 0]]), columns=["cirtime", "cir"])
            cirDF["cir"] = cirDF["cir"].astype(float)
            reset_values(cirStart) 

    if "Scheduled Basal Rate" in a:
        if previousType != a:
            set_values(sbrDF,  "sbr", sbrLabel,
                       sbrMin, sbrMax, sbrStep, '.3f')
        
        if "Add" in d:
            df, sbrDF = add_values(df, sbrDF, "sbr")      

        if "Reset" in d:
            sbrDF = pd.DataFrame(np.array([["12:00 AM", 0]]), columns=["sbrtime", "sbr"])
            sbrDF["sbr"] = sbrDF["sbr"].astype(float)
            reset_values(sbrStart)
       
    if "Temp Basal Events" in a:
        if previousType != a:
            set_values(abrDF, "abr", abrLabel, abrMin, abrMax, abrStep, '.3f')
            durationWid1Label.value = "Duration (min)"
            durationWid1.options = np.arange(0, 24*60, 5)
            durationWid1.value = 0
            durationWid1Box.layout.visibility="visible"
        
        if "Add" in d:
            df, abrDF = add_valuesAndDuration(df, abrDF, "abr")      

        if "Reset" in d:
            abrDF = pd.DataFrame(np.array([["12:00 AM", 0, 0]]), columns=["abrtime", "abr", "abrDuration"])
            abrDF["abr"] = abrDF["abr"].astype(float)
            durationWid1.value = 0
            reset_values(abrStart)

    if "Bolus Events" in a:
        if previousType != a:
            set_values(insulinDF, "insulin", "Bolus Events (Insulin & Carb Amounts)", insulinMin, insulinMax, insulinStep, '.3f')
            set_values2(carbDF, "carb", carbMin, carbMax, carbStep, '.0f')
            durationWid1Label.value = ""
            durationWid1.options = ["Insulin Only", "Carb Only", "Insulin & Carb"]
            durationWid1.value = "Insulin & Carb"
            durationWid2.options = ["highGI", "medGI", "lowGI"]
            durationWid2.value = "medGI"
            timeWid2.layout.visibility="visible"
            valueWid2.layout.visibility="visible"
            durationWid1Box.layout.visibility="visible"
            durationWid2.layout.visibility="visible"
            valueWid1.description="insulin (U)"
            valueWid2.description="carbs (g)"

        if (("Insulin" in durationWid1.value) & ("Carb" not in durationWid1.value)):
            timeWid2.disabled = True
            valueWid2.disabled = True
            durationWid2.disabled = True

        elif (("Carb" in durationWid1.value) & ("Insulin" not in durationWid1.value)):
            timeWid1.disabled = True
            valueWid1.disabled = True
            
        if "Insulin" in durationWid1.value:
            timeWid1.disabled = False
            valueWid1.disabled = False

        if "Carb" in durationWid1.value:
            timeWid2.disabled = False
            valueWid2.disabled = False
            durationWid2.disabled = False

        if (("Insulin" in durationWid1.value) & ("Carb" in durationWid1.value)):
            timeWid2.value = timeWid1.value
            valueWid1.value = valueWid2.value / df.loc[df["dateTime"] == pd.to_datetime(timeWid1.value), "cir"]
            timeWid2.disabled = True
            valueWid1.disabled = True
                    
        if "Add" in d:
            if "Insulin" in durationWid1.value:
                df, insulinDF = add_insulinValues(df, insulinDF, "insulin") 
                
            if "Carb" in durationWid1.value:
                df, carbDF = add_carbValues(df, carbDF, "carb")

        if "Reset" in d:
            if "Insulin" in durationWid1.value:
                insulinDF = pd.DataFrame(np.array([["12:00 AM", 0, insulinModel]]),
                          columns=["insulintime", "insulin", "insulinModel"])
                insulinDF["insulin"] = insulinDF["insulin"].astype(float)

            if "Carb" in durationWid1.value:
                carbDF = pd.DataFrame(np.array([["12:00 AM", 0, "medGI"]]),
                          columns=["carbtime", "carb", "carbModel"])
                carbDF["carb"] = carbDF["carb"].astype(float)
                durationWid2.value = "medGI"
                
            if (("Insulin" in durationWid1.value) & ("Carb" in durationWid1.value)):
                reset_values(insulinStart, carbStart)
            elif "Insulin" in durationWid1.value:
                reset_values(insulinStart, np.nan)
            else:
                reset_values(np.nan, carbStart)

    # plot the settings and events
    ax0.plot(df["minutesRelativeToNow"], df["isf"], 
        linestyle=":", color="#f6cc89", lw=5, alpha=0.75, label=isfLabel)

    ax0.plot(df["minutesRelativeToNow"], df["cir"],
        linestyle="-.", color="#83d754", lw=3, alpha=0.75, label=cirLabel)

    ax0.plot(df["minutesRelativeToNow"], df["sbr"],
        linestyle="--", color="#f09a37", lw=3, alpha=0.75, label=sbrLabel)

    ax0.plot(df["minutesRelativeToNow"], df["abr"],
        linestyle="-", color="#ff8800", lw=4, alpha=0.75, label=abrLabel)
    
    # handle the case where temp basal is 0
    ax0.plot(df.loc[df["abr"]== 0, "minutesRelativeToNow"], df.loc[df["abr"]==0, "abr"]+0.015,
        linestyle="-", color="#F05237", lw=4, alpha=0.75, label="Suspend Event")   

    # add in insulin and carb events
    if sum(df["insulin"]>0) > 0:
        ax0.plot(df["minutesRelativeToNow"], df["insulin"], alpha=0.75,
                 linestyle="None", marker='v', markersize=12, color="#f09a37", label=insulinLabel)
    else:
        ax0.plot(-10, -10, alpha=0.75, linestyle="None", marker='v', markersize=12, color="#f09a37", label=insulinLabel)    

    if sum(df["carb"]>0) > 0:
        ax0.plot(df["minutesRelativeToNow"], df["carb"], alpha=0.75,
                 linestyle="None", marker='^', markersize=12, color="#83d754", label=carbLabel)
    else:
        ax0.plot(-10, -10, alpha=0.75, linestyle="None", marker='^', markersize=12, color="#83d754", label=carbLabel)

    # format the legend
    leg = ax0.legend(edgecolor="black", loc="upper right")
    for text in leg.get_texts():
        text.set_color('#606060')
        text.set_weight('normal')
        text.set_fontsize(legendFontSize)

    # %% RUN THE SIMULATION

    # %% define the model
    deltaBgEquation = "ISF * ((ACV/CIR) - AIV)"
    deltaBG = create_deltaBG_functions(deltaBgEquation)

    # %% load in activity curve data
    insulinActivityCurves = pd.read_csv("all-iac-models.csv", low_memory=False)
    carbActivityCurves = pd.read_csv("all-cac-models.csv", low_memory=False)
    insulinActivityCurve = insulinActivityCurves[insulinModel].values

    # %% first create a continuguous time series
    timeStepSize = 5  # minutes
    effectLength = 12  # hours, the display with be +/- this length
    timeSeriesLength = effectLength * 3  # hours
    roundedCurrentTime = pd.to_datetime(currentTime).round(str(timeStepSize) + "min")
    startTime = roundedCurrentTime + pd.Timedelta(-timeSeriesLength, unit="h")
    endTime = roundedCurrentTime + pd.Timedelta(timeSeriesLength, unit="h")
    rng = pd.date_range(startTime, endTime, freq=(str(timeStepSize) + "min"))
    data = pd.DataFrame(rng, columns=["dateTime"])
    # create an initial time series that is 3 times the length of simulation plot
    data["minutesRelativeToNow"] = np.arange(-timeSeriesLength*60,
                                             timeSeriesLength*60 + timeStepSize,
                                             timeStepSize)
    data = pd.merge(data, df.drop(columns="minutesRelativeToNow"), how="outer", on="dateTime")
    data.cir.fillna(method='bfill', inplace=True)
    data.cir.fillna(method='ffill', inplace=True)
    data.sbr.fillna(method='bfill', inplace=True)
    data.sbr.fillna(method='ffill', inplace=True)
    data.isf.fillna(method='bfill', inplace=True)
    data.isf.fillna(method='ffill', inplace=True)
    data.insulin.fillna(0, inplace=True)
    data.carb.fillna(0, inplace=True)

    # define the effective basal rate (U/hr)
    data["ebr"] = (data.abr - data.sbr)
    # assume that any basal that is NOT the scheduled basal or suspend is the scheduled basal rate
    data["ebr"].fillna(0, inplace=True)
    # define the effective basal rate (U/5min)
    data["ebi"] = data["ebr"] / 12

    # % merge basal and bolus insulin amounts together
    data["tia"] = data["ebi"] + data["insulin"]

    # % calculate the insulin activity
    data["aiv"] = 0
    tempDf = data.loc[data["tia"] != 0, ["tia", "dateTime"]]
    for amount, ind in zip(tempDf.tia, tempDf.index):
        aiv = pd.DataFrame(np.array([amount * insulinActivityCurve]).reshape(-1, 1),
                           index=np.arange(ind, (ind + len(insulinActivityCurve))), columns=["aiv"])
        data.loc[ind:(ind + len(insulinActivityCurve)), ["aiv"]] = data["aiv"].add(aiv["aiv"], fill_value=0)

    # % calculate the carb activity
    data["acv"] = 0
    tempDf = data.loc[data["carb"] > 0, ["carb", "carbModel", "dateTime"]]
    for amount, model, ind in zip(tempDf["carb"], tempDf["carbModel"], tempDf.index):
        acv = pd.DataFrame(np.array([amount * carbActivityCurves[model]]).reshape(-1, 1),
                           index=np.arange(ind, (ind + len(insulinActivityCurve))), columns=["acv"])
        data.loc[ind:(ind + len(insulinActivityCurve)), ["acv"]] = data["acv"].add(acv["acv"], fill_value=0)

    # % window the contiguous data to the effect length
    startTime = roundedCurrentTime + pd.Timedelta(-effectLength, unit="h")
    endTime = roundedCurrentTime + pd.Timedelta(effectLength, unit="h")
    rng = pd.date_range(startTime, endTime, freq=(str(timeStepSize) + "min"))
    df2 = pd.DataFrame(rng, columns=["dateTime"])
    df2 = pd.merge(df2, data, how="left", on="dateTime")

    # % make the prediction with the new data
    df2["bg"] = startingBGLevel
    df2["dBG"] = deltaBG(ISF=df2.isf.values,
                        CIR=df2.cir.values,
                        ACV=df2.acv.values,
                        AIV=df2.aiv.values)

    df2["simulatedBG"] = df2["bg"] + df2["dBG"].cumsum()
    df2.loc[df2["simulatedBG"] <= 40, "simulatedBG"] = 40
    df2.loc[df2["simulatedBG"] >= 400, "simulatedBG"] = 400
 
    # plot simulated cgm
    ax.scatter(df2["minutesRelativeToNow"], df2["simulatedBG"], s=10, color="#31B0FF", label="Predicted BG Level")
    
    # plot the starting bg time
    ax.plot(df2.minutesRelativeToNow[0], startingBGLevel,
            marker='*', markersize=20, color="#31B0FF", markeredgecolor="black", alpha=0.5,
            ls="None", label="Starting BG =  %d" % (round(startingBGLevel)))

    # plot the current time
    valueCurrentTime = df2.loc[df2["minutesRelativeToNow"] == 0, "simulatedBG"].values[0]
    ax.plot(0, valueCurrentTime,
            marker='*', markersize=20, color=coord_color, markeredgecolor="black", alpha=0.5,
            ls="None", label="Current Time BG =  %d" % (round(valueCurrentTime)))
    
    # format the legend
    leg = ax.legend(scatterpoints=3, edgecolor="black", loc="upper right")
    for text in leg.get_texts():
        text.set_color('#606060')
        text.set_weight('normal')
        text.set_fontsize(legendFontSize)

    # %% plot the effective basal rate
    ax5.plot(df2["minutesRelativeToNow"],
             df2["ebr"], linestyle="-", color="#f09a37", lw=3, label="Effective Basal Rate (U/hr)")

    # fill in temp basals and suspends
    ax5.fill_between(df2["minutesRelativeToNow"], df2["ebr"], color="#f6cc89")

    # %% plot the active effects
    # plot the active insulin
    activeInsulin = -df2["aiv"]*df2["isf"]
    ax7.plot(df2["minutesRelativeToNow"], activeInsulin,
             linestyle="--", color="#f09a37", 
             lw=3, alpha=0.5, label=r'Active Insulin $\frac{mg/dL}{5min}$')

    ax7.fill_between(df2["minutesRelativeToNow"], activeInsulin, 0, color="#f6cc89", alpha=0.5)

    # plot the active carbs
    activeCarbs = df2["acv"]*df2["isf"]/df2["cir"]
    ax7.plot(df2["minutesRelativeToNow"], activeCarbs,
             linestyle="-.", color="#83d754",
             lw=3, alpha=0.5, label=r'Active Carbs $\frac{mg/dL}{5min}$')

    ax7.fill_between(df2["minutesRelativeToNow"], activeCarbs, 0, color="#bdeaa3", alpha=0.5)

    leg = ax7.legend(edgecolor="black", loc="upper right")
    for text in leg.get_texts():
        text.set_color('#606060')
        text.set_weight('normal')
        text.set_fontsize(legendFontSize)

    previousType = dataTypeWid.value
    
    # %% capture all of the settings for saving
    allSettings = pd.merge(isfDF, cirDF, how="outer", on="dateTime")
    allSettings = pd.merge(allSettings, sbrDF, how="outer", on="dateTime")
    allSettings = pd.merge(allSettings, abrDF, how="outer", on="dateTime")
    allSettings = pd.merge(allSettings, insulinDF, how="outer", on="dateTime")
    allSettings = pd.merge(allSettings, carbDF, how="outer", on="dateTime")
    
    #plt.subplots_adjust(hspace=0.5)

    plt.show()
    
out = widgets.interactive_output(f, {'a': dataTypeWid,
                                     'b': timeWid1,
                                     'c': valueWid1,
                                     'd': addResetButton,
                                     'g': timeWid2,
                                     'h': durationWid1,
                                     'j': durationWid2,
                                     'k': valueWid2
                                    }
)

display(inputBox, out)

# TODOs:
# * [] allow user to select the insulin model, and starting BG
# * [] allow user to save the input 
# * [] allow the user to load in settings from saved file
# * [] allow user to pull in data from tidepool
# * [] fix temp basal and bolus events so there isn't a big delay when switching


Output()

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
Name: abr, dtype: float64

In [14]:
isfMin

10